In [1]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3.2", model_provider="ollama")

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! It's nice to meet you. Is there something I can help you with, or would you like to chat for a bit?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-26T23:19:43.95176Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11935269700, 'load_duration': 7470851700, 'prompt_eval_count': 30, 'prompt_eval_duration': 2163329700, 'eval_count': 30, 'eval_duration': 2263572500, 'model_name': 'llama3.2'}, id='run--c333b98f-7fa4-46ed-a91a-80c306ea8ad8-0', usage_metadata={'input_tokens': 30, 'output_tokens': 30, 'total_tokens': 60})

In [7]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-26T23:20:08.4715232Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1260076400, 'load_duration': 64487000, 'prompt_eval_count': 55, 'prompt_eval_duration': 639346000, 'eval_count': 6, 'eval_duration': 549895900, 'model_name': 'llama3.2'}, id='run--cdce345d-84e8-4d9b-8b69-2880c514b0af-0', usage_metadata={'input_tokens': 55, 'output_tokens': 6, 'total_tokens': 61})

In [9]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

In [13]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with, or would you like to chat for a bit?


In [19]:
config = {"configurable": {"thread_id": "abc123"}}
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name was Bob, but I'm not actually reading any information from our conversation to confirm it. In this chat, we just started, and you introduced yourself as Bob. If you'd like to share more about yourself, I'm here to listen!


In [21]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm not capable of knowing your personal identity, including your name. Our conversation just started, and I don't have any prior information about you. If you'd like to share your name with me, I'd be happy to chat with you!


In [23]:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

In [27]:
# Async invocation:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Bob! It's great to have you here. Is there anything on your mind that you'd like to talk about, or would you like some conversation starters? I'm all ears (or rather, all text).


<h1>Prompt templates</h1>

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [33]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [35]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer lookin' fer some chat, eh? Well, matey Jim, I be happy to oblige! What be bringin' ye to these fair waters o' conversation today? Got a question or just want to have a swashbucklin' good time? Fire away, me hearty!


In [37]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, don't ye worry about that, Jim! Yer name be right in front o' ye, matey - J-I-M! No need fer confusion here. Ye just be forgettin' it fer a sec, savvy?


In [39]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [41]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [43]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola Bob! (Hello, Bob!) ¿En qué puedo ayudarte hoy? (How can I help you today?)


In [45]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. (Your name is Bob.)
